In [89]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(
    # model="hf.co/teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf:Q4_K_M",
    # model="llama3.2:latest",  # 사용할 언어 모델을 지정합니다.
    model="mistral:latest",  # 사용할 언어 모델을 지정합니다.
    # model="mixtral:latest",  # 사용할 언어 모델을 지정합니다.
    # model="eeve-ko-Q5-KM:latest",
    # format="json",  # 입출력 형식을 JSON으로 설정합니다.
    temperature=0.3,
)


In [90]:
# 실시간 출력을 위한 import
from langchain_teddynote.messages import stream_response

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

# llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = prompt | llm

answer = chain.stream({"email_conversation": email_conversation})

output = stream_response(answer, return_output=True)


1. 제목: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안
    - 내용: 김철수(상무)가 이은채 대리님에게 "ZENESIS" 모델에 대한 상세한 브로슈어를 요청하고, 협력을 위해 다음 주 화요일 오전 10시에 미팅을 제안합니다.
    - 필요 정보: 기술 사양, 배터리 성능, 디자인 측면에 대한 "ZENESIS" 모델의 정보가 필요합니다.
    - 목적: 이해관계자(Teddy International)와 바이크코퍼레이션이 "ZENESIS" 모델을 유통할 수 있는 방법과 전략에 대해 토론하고, 협력하기 위함.

In [72]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

# instruction 을 출력합니다.
# print(parser.get_format_instructions())

prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.
  - 문자열 내부에 큰따옴표(")가 포함되면 반드시 백슬래시(\\")로 이스케이프하세요.
---

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

# chain 을 생성합니다.
chain = prompt | llm

# chain 을 실행하고 결과를 출력합니다.
response = chain.stream(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 JSON 형태로 출력됩니다.
output = stream_response(response, return_output=True)
# output
# PydanticOutputParser 를 사용하여 결과를 파싱합니다.
structured_output = parser.parse(output)
print(structured_output)


이메일 내용을 JSON 형식으로 변환하여 주세요.

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션의 김철수 상무는 귀사(teddyinternational.me)에게 'ZENESIS' 신규 자전거에 대한 상세한 정보를 요청하며, 다음 주 화요일 오전 10시에 미팅을 제안합니다.",
  "date": "1월 15일"
}
```person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary="바이크코퍼레이션의 김철수 상무는 귀사(teddyinternational.me)에게 'ZENESIS' 신규 자전거에 대한 상세한 정보를 요청하며, 다음 주 화요일 오전 10시에 미팅을 제안합니다." date='1월 15일'


In [73]:
# 출력 파서를 추가하여 전체 체인을 재구성합니다.
# chain = prompt | llm | parser

from langchain_core.output_parsers import StrOutputParser
# 전체 체인을 문자열 → JSON 구조화로 연결
chain = prompt | llm | StrOutputParser() | parser

# chain 을 실행하고 결과를 출력합니다.
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

# 결과는 EmailSummary 객체 형태로 출력됩니다.
response

EmailSummary(person='김철수 (chulsoo.kim@bikecorporation.me)', email='이은채@teddyinternational.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션의 김철수 상무가 귀사에게 'ZENESIS' 모델에 대한 상세한 정보를 요청하고, 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다.", date='2023년 1월 15일 오전 10시')

In [81]:
# Comma Output Parser
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()
# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    # 주제에 대한 다섯 가지를 나열하라는 템플릿
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],  # 입력 변수로 'subject' 사용
    # 부분 변수로 형식 지침 사용
    partial_variables={"format_instructions": format_instructions},
)

# ChatOpenAI 모델 초기화
# model = ChatOpenAI(temperature=0)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | llm | output_parser

# "대한민국 관광명소"에 대한 체인 호출 실행
chain.invoke({"subject": "대한민국 관광명소"})

# 스트림을 순회합니다.
# for s in chain.stream({"subject": "대한민국 관광명소"}):
#     print(s)  # 스트림의 내용을 출력합니다.



['세종대왕십리공원', '부산타워', '제주도 성산지구', '강릉동해마рин라이프파크', '전라남도 순천향국립공원']

In [85]:
## Structured Output Parser

from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 사용자의 질문에 대한 답변
response_schemas = [
    ResponseSchema(name="answer", description="사용자의 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이여야 합니다.",
    ),
]
# 응답 스키마를 기반으로 한 구조화된 출력 파서 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 출력 형식 지시사항을 파싱합니다.
format_instructions = output_parser.get_format_instructions()
format_instructions

prompt = PromptTemplate(
    # 사용자의 질문에 최대한 답변하도록 템플릿을 설정합니다.
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    # 입력 변수로 'question'을 사용합니다.
    input_variables=["question"],
    # 부분 변수로 'format_instructions'을 사용합니다.
    partial_variables={"format_instructions": format_instructions},
)

# model = ChatOpenAI(temperature=0)  # ChatOpenAI 모델 초기화
chain = prompt | llm | output_parser  # 프롬프트, 모델, 출력 파서를 연결

# 대한민국의 수도가 무엇인지 질문합니다.
chain.invoke({"question": "대한민국의 수도는 어디인가요?"})


{'answer': '대한민국의 수도는 서울입니다.',
 'source': 'https://ko.wikipedia.org/wiki/%EC%84%9C%EC%9A%B8'}

In [91]:
# JSON Output Parser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# OpenAI 객체를 생성합니다.
# model = ChatOpenAI(temperature=0, model_name="gpt-4o")

# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

chain = prompt | llm | parser  # 체인을 구성합니다.

chain.invoke({"question": question})  # 체인을 호출하여 쿼리 실행



{'description': '지구 온난화는 지구 평균 기온이 1900년대에 비해 +1.5도 이상 올라가는 현상으로, 자연적인 변화와 함께 사람의 활동으로 인한 탄소 排출로 주로 인정됩니다.\n온난화에 따른 영향은 다양합니다. 예를 들어, 강수기 변동, 해양 생태계 파괴, 지방 물자원 저하, 농업 생산력 저하 등이 있습니다.\n또한, 온난화에 따른 민간 사고와 국가 보안 위협으로까지 확대되고 있습니다.',
 'hashtags': '#ClimateChange #GlobalWarming'}

In [92]:
# 질의 작성
question = "지구 온난화에 대해 알려주세요. 온난화에 대한 설명은 `description`에, 관련 키워드는 `hashtags`에 담아주세요."

# JSON 출력 파서 초기화
parser = JsonOutputParser()

# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

# 지시사항을 프롬프트에 주입합니다.
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 프롬프트, 모델, 파서를 연결하는 체인 생성
chain = prompt | llm | parser

# 체인을 호출하여 쿼리 실행
response = chain.invoke({"question": question})

# 출력을 확인합니다.
print(response)


{'description': '지구 온난화란 지구의 평균 기후 변화로, 인간의 활동으로 인한 탄소 배출이 가장 큰 원인입니다. 이는 일상생활에서 차량 운행, 전기 발전 등을 통해 생성되며, 대기 질환, 물자원 감소, 생태계 파괴 등의 문제를 야기합니다. 온난화에 대응하기 위한 방안으로는 탄소 배출을 줄이고, 지속가능한 발전 시스템, 재활용 및 재사용 등의 행동이 필요합니다.', 'hashtags': ['climatechange', 'globalwarming', 'sustainability', 'renewableenergy', 'reducecarbonfootprint']}
